## Spacial patterns of crimes and their drivers: evidence from London 

In [1]:
import pandas as pd
from pathlib import Path
from stargazer.stargazer import Stargazer
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt

### Structure of the logic in the article

In general, the theme is what are the contributors of crime at LSOA level. 

it could be spacial reasons. for instance, via neighbourhood spiill-over effects. or it could be due to individual-specific factors. So I explore both possiblities in the following way. 

1. first, empircally observe spacial correlation between lsoas within London. I found that there is no strong correlation. 
  - but many people, or many discussion may have thought that crime by in large is a spacial issue, and has strong spacial patterns. for instance, people believe that crime is spacially contagious, i.e. if crime in neighbourhoods is high, more likely have crimes in my study areas. 

2. second, ask what are the drivers of crime rates at the individual lsoa level. to put it differently, what lsoa specific characteristics can explain the heterogeneity in crime rates, even between regions that are spacially close.

  - some of the variables are directly observable at lsoa level.
  - some of the variables are only available at bourough level.
  - both types of variations can be used to explain the differences in crime rates. 
  - but if I want to control for bourough fixed effects, those variables that are only available at bourough levels are naturally dropped becasue the bourough fixed effects already contain them. 
  

### Crime data 

In [2]:
crime_ipath = Path("../data/london-data-main/data/crime/")
historic_fp = crime_ipath/"MPS LSOA Level Crime (Historical).csv"
historic_df = pd.read_csv(historic_fp)
#historic_df.columns

In [3]:
### get average crime rate of each place 
## compute summary stats across time 
#only count 2011

dates2011 = [date for date in historic_df.columns if '2011' in date]

historic_df['crime2011'] = historic_df[dates2011].sum(axis=1)

#historic_df['sample_average_crime'] = historic_df[all_dates].mean(axis=1) ## or use median or sum 

historic_df = historic_df[['LSOA Code','LSOA Name','Borough','Major Category','Minor Category','crime2011']]

In [4]:
historic_df.columns

Index(['LSOA Code', 'LSOA Name', 'Borough', 'Major Category', 'Minor Category',
       'crime2011'],
      dtype='object')

In [5]:
historic_df[['LSOA Code', 'LSOA Name','Borough', 'Major Category', 'Minor Category', 'crime2011']]

,LSOA Code,LSOA Name,Borough,Major Category,Minor Category,crime2011
0,E01000006,Barking and Dagenham 016A,Barking and Dagenham,Arson and Criminal Damage,Arson,0
1,E01000006,Barking and Dagenham 016A,Barking and Dagenham,Arson and Criminal Damage,Criminal Damage,4
2,E01000006,Barking and Dagenham 016A,Barking and Dagenham,Burglary,Burglary Business and Community,1
3,E01000006,Barking and Dagenham 016A,Barking and Dagenham,Burglary,Domestic Burglary,21
4,E01000006,Barking and Dagenham 016A,Barking and Dagenham,Drug Offences,Drug Trafficking,1
...,...,...,...,...,...,...
147546,E01033746,Greenwich 038E,Greenwich,Vehicle Offences,Interfering with a Motor Vehicle,1
147547,E01033746,Greenwich 038E,Greenwich,Vehicle Offences,Theft from a Motor Vehicle,11
147548,E01033746,Greenwich 038E,Greenwich,Vehicle Offences,Theft or Taking of a Motor Vehicle,1
147549,E01033746,Greenwich 038E,Greenwich,Violence Against the Person,Violence with Injury,3


In [6]:
property2011 = [crime for crime in historic_df['Major Category'] if 'Burglary' or 'Theft' in crime]
property2011

['Arson and Criminal Damage',
 'Arson and Criminal Damage',
 'Burglary',
 'Burglary',
 'Drug Offences',
 'Drug Offences',
 'Miscellaneous Crimes Against Society',
 'Miscellaneous Crimes Against Society',
 'Miscellaneous Crimes Against Society',
 'Miscellaneous Crimes Against Society',
 'Miscellaneous Crimes Against Society',
 'Miscellaneous Crimes Against Society',
 'Miscellaneous Crimes Against Society',
 'Possession of Weapons',
 'Possession of Weapons',
 'Public Order Offences',
 'Public Order Offences',
 'Public Order Offences',
 'Robbery',
 'Theft',
 'Theft',
 'Theft',
 'Vehicle Offences',
 'Vehicle Offences',
 'Vehicle Offences',
 'Violence Against the Person',
 'Violence Against the Person',
 'Arson and Criminal Damage',
 'Arson and Criminal Damage',
 'Burglary',
 'Burglary',
 'Drug Offences',
 'Drug Offences',
 'Miscellaneous Crimes Against Society',
 'Miscellaneous Crimes Against Society',
 'Miscellaneous Crimes Against Society',
 'Miscellaneous Crimes Against Society',
 'Misc

In [7]:
## reshape the data 
historic_df['crime_type'] = historic_df['Major Category']+': '+historic_df['Minor Category']

rehistoric_df = pd.pivot(historic_df, 
                       index=['LSOA Code', 'LSOA Name', 'Borough'], 
                       columns='crime_type', 
                       values='crime2011')

rehistoric_df =rehistoric_df.reset_index()



In [8]:
rehistoric_df

crime_type,LSOA Code,LSOA Name,Borough,Arson and Criminal Damage: Arson,Arson and Criminal Damage: Criminal Damage,Burglary: Burglary Business and Community,Burglary: Domestic Burglary,Drug Offences: Drug Trafficking,Drug Offences: Possession of Drugs,Miscellaneous Crimes Against Society: Absconding from Lawful Custody,...,Theft: Other Theft,Theft: Shoplifting,Theft: Theft from Person,Vehicle Offences: Aggravated Vehicle Taking,Vehicle Offences: Interfering with a Motor Vehicle,Vehicle Offences: Theft from a Motor Vehicle,Vehicle Offences: Theft or Taking of a Motor Vehicle,Violence Against the Person: Homicide,Violence Against the Person: Violence with Injury,Violence Against the Person: Violence without Injury
0,E01000006,Barking and Dagenham 016A,Barking and Dagenham,0.0,4.0,1.0,21.0,1.0,9.0,NaN,...,7.0,NaN,5.0,NaN,2.0,16.0,13.0,NaN,10.0,7.0
1,E01000007,Barking and Dagenham 015A,Barking and Dagenham,2.0,39.0,5.0,8.0,2.0,50.0,0.0,...,49.0,3.0,28.0,1.0,0.0,28.0,7.0,0.0,66.0,53.0
2,E01000008,Barking and Dagenham 015B,Barking and Dagenham,1.0,14.0,4.0,9.0,2.0,18.0,NaN,...,31.0,4.0,2.0,NaN,1.0,25.0,14.0,NaN,12.0,20.0
3,E01000009,Barking and Dagenham 016B,Barking and Dagenham,3.0,34.0,9.0,14.0,2.0,54.0,0.0,...,29.0,12.0,16.0,0.0,4.0,23.0,13.0,0.0,44.0,42.0
4,E01000010,Barking and Dagenham 015C,Barking and Dagenham,3.0,69.0,33.0,18.0,0.0,102.0,NaN,...,169.0,283.0,80.0,0.0,1.0,46.0,13.0,0.0,96.0,122.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4824,E01033742,Greenwich 007F,Greenwich,1.0,9.0,1.0,5.0,0.0,2.0,NaN,...,7.0,NaN,1.0,0.0,1.0,7.0,2.0,NaN,7.0,7.0
4825,E01033743,Greenwich 002H,Greenwich,1.0,22.0,3.0,3.0,1.0,8.0,NaN,...,110.0,52.0,6.0,1.0,2.0,15.0,6.0,NaN,23.0,29.0
4826,E01033744,Greenwich 007G,Greenwich,2.0,12.0,5.0,8.0,0.0,3.0,NaN,...,9.0,1.0,1.0,NaN,0.0,13.0,1.0,NaN,16.0,13.0
4827,E01033745,Greenwich 002I,Greenwich,1.0,22.0,4.0,7.0,1.0,2.0,NaN,...,23.0,0.0,0.0,0.0,1.0,11.0,8.0,NaN,22.0,10.0


In [9]:
## count total crime of all types 

all_crime = [crime for crime in list(rehistoric_df.columns) if crime not in ['LSOA Code','LSOA Name','Borough']]

rehistoric_df['all_crime'] = rehistoric_df[all_crime].sum(axis=1, numeric_only=True)

In [10]:
rehistoric_df2 = rehistoric_df[['LSOA Code','LSOA Name','Borough','all_crime']]
rehistoric_df2

crime_type,LSOA Code,LSOA Name,Borough,all_crime
0,E01000006,Barking and Dagenham 016A,Barking and Dagenham,109.0
1,E01000007,Barking and Dagenham 015A,Barking and Dagenham,426.0
2,E01000008,Barking and Dagenham 015B,Barking and Dagenham,179.0
3,E01000009,Barking and Dagenham 016B,Barking and Dagenham,351.0
4,E01000010,Barking and Dagenham 015C,Barking and Dagenham,1202.0
...,...,...,...,...
4824,E01033742,Greenwich 007F,Greenwich,65.0
4825,E01033743,Greenwich 002H,Greenwich,301.0
4826,E01033744,Greenwich 007G,Greenwich,95.0
4827,E01033745,Greenwich 002I,Greenwich,133.0


In [11]:
#only shows the property crime in dataset
historic_dfp = historic_df[(historic_df['Major Category'] == 'Burglary') | (historic_df['Major Category'] == 'Theft') | (historic_df['Major Category'] == 'Robbery')]
#sum the data
new_df = historic_dfp.groupby('LSOA Code')['crime2011'].sum().reset_index()
new_df = new_df.rename(columns={'crime2011': 'PropertyCrime'})
new_df

,LSOA Code,PropertyCrime
0,E01000006,43
1,E01000007,135
2,E01000008,61
3,E01000009,109
4,E01000010,654
...,...,...
4824,E01033742,23
4825,E01033743,183
4826,E01033744,31
4827,E01033745,46


In [12]:
#merge the property data into the orginal data
rehistoric_df3 = pd.merge(rehistoric_df,
                    new_df,
                   left_on ='LSOA Code',
                   right_on='LSOA Code',
                   how='inner')
rehistoric_df3

,LSOA Code,LSOA Name,Borough,Arson and Criminal Damage: Arson,Arson and Criminal Damage: Criminal Damage,Burglary: Burglary Business and Community,Burglary: Domestic Burglary,Drug Offences: Drug Trafficking,Drug Offences: Possession of Drugs,Miscellaneous Crimes Against Society: Absconding from Lawful Custody,...,Theft: Theft from Person,Vehicle Offences: Aggravated Vehicle Taking,Vehicle Offences: Interfering with a Motor Vehicle,Vehicle Offences: Theft from a Motor Vehicle,Vehicle Offences: Theft or Taking of a Motor Vehicle,Violence Against the Person: Homicide,Violence Against the Person: Violence with Injury,Violence Against the Person: Violence without Injury,all_crime,PropertyCrime
0,E01000006,Barking and Dagenham 016A,Barking and Dagenham,0.0,4.0,1.0,21.0,1.0,9.0,NaN,...,5.0,NaN,2.0,16.0,13.0,NaN,10.0,7.0,109.0,43
1,E01000007,Barking and Dagenham 015A,Barking and Dagenham,2.0,39.0,5.0,8.0,2.0,50.0,0.0,...,28.0,1.0,0.0,28.0,7.0,0.0,66.0,53.0,426.0,135
2,E01000008,Barking and Dagenham 015B,Barking and Dagenham,1.0,14.0,4.0,9.0,2.0,18.0,NaN,...,2.0,NaN,1.0,25.0,14.0,NaN,12.0,20.0,179.0,61
3,E01000009,Barking and Dagenham 016B,Barking and Dagenham,3.0,34.0,9.0,14.0,2.0,54.0,0.0,...,16.0,0.0,4.0,23.0,13.0,0.0,44.0,42.0,351.0,109
4,E01000010,Barking and Dagenham 015C,Barking and Dagenham,3.0,69.0,33.0,18.0,0.0,102.0,NaN,...,80.0,0.0,1.0,46.0,13.0,0.0,96.0,122.0,1202.0,654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4824,E01033742,Greenwich 007F,Greenwich,1.0,9.0,1.0,5.0,0.0,2.0,NaN,...,1.0,0.0,1.0,7.0,2.0,NaN,7.0,7.0,65.0,23
4825,E01033743,Greenwich 002H,Greenwich,1.0,22.0,3.0,3.0,1.0,8.0,NaN,...,6.0,1.0,2.0,15.0,6.0,NaN,23.0,29.0,301.0,183
4826,E01033744,Greenwich 007G,Greenwich,2.0,12.0,5.0,8.0,0.0,3.0,NaN,...,1.0,NaN,0.0,13.0,1.0,NaN,16.0,13.0,95.0,31
4827,E01033745,Greenwich 002I,Greenwich,1.0,22.0,4.0,7.0,1.0,2.0,NaN,...,0.0,0.0,1.0,11.0,8.0,NaN,22.0,10.0,133.0,46


In [13]:
#only shows the antisocial behaviour crime in dataset
historic_dfa = historic_df[(historic_df['Major Category'] == 'Drug Offences') | (historic_df['Major Category'] == 'Miscellaneous Crimes Against Society') | (historic_df['Major Category'] == 'Public Order Offences')]
#sum the data
new_df1 = historic_dfa.groupby('LSOA Code')['crime2011'].sum().reset_index()
new_df1 = new_df1.rename(columns={'crime2011': 'antiCrime'})
new_df1

,LSOA Code,antiCrime
0,E01000006,14
1,E01000007,89
2,E01000008,30
3,E01000009,77
4,E01000010,186
...,...,...
4824,E01033742,7
4825,E01033743,18
4826,E01033744,7
4827,E01033745,12


In [14]:
#merge the antisocial behaviour crime data into the orginal data
rehistoric_df4 = pd.merge(rehistoric_df3,
                    new_df1,
                   left_on ='LSOA Code',
                   right_on='LSOA Code',
                   how='inner')
rehistoric_df4

,LSOA Code,LSOA Name,Borough,Arson and Criminal Damage: Arson,Arson and Criminal Damage: Criminal Damage,Burglary: Burglary Business and Community,Burglary: Domestic Burglary,Drug Offences: Drug Trafficking,Drug Offences: Possession of Drugs,Miscellaneous Crimes Against Society: Absconding from Lawful Custody,...,Vehicle Offences: Aggravated Vehicle Taking,Vehicle Offences: Interfering with a Motor Vehicle,Vehicle Offences: Theft from a Motor Vehicle,Vehicle Offences: Theft or Taking of a Motor Vehicle,Violence Against the Person: Homicide,Violence Against the Person: Violence with Injury,Violence Against the Person: Violence without Injury,all_crime,PropertyCrime,antiCrime
0,E01000006,Barking and Dagenham 016A,Barking and Dagenham,0.0,4.0,1.0,21.0,1.0,9.0,NaN,...,NaN,2.0,16.0,13.0,NaN,10.0,7.0,109.0,43,14
1,E01000007,Barking and Dagenham 015A,Barking and Dagenham,2.0,39.0,5.0,8.0,2.0,50.0,0.0,...,1.0,0.0,28.0,7.0,0.0,66.0,53.0,426.0,135,89
2,E01000008,Barking and Dagenham 015B,Barking and Dagenham,1.0,14.0,4.0,9.0,2.0,18.0,NaN,...,NaN,1.0,25.0,14.0,NaN,12.0,20.0,179.0,61,30
3,E01000009,Barking and Dagenham 016B,Barking and Dagenham,3.0,34.0,9.0,14.0,2.0,54.0,0.0,...,0.0,4.0,23.0,13.0,0.0,44.0,42.0,351.0,109,77
4,E01000010,Barking and Dagenham 015C,Barking and Dagenham,3.0,69.0,33.0,18.0,0.0,102.0,NaN,...,0.0,1.0,46.0,13.0,0.0,96.0,122.0,1202.0,654,186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4824,E01033742,Greenwich 007F,Greenwich,1.0,9.0,1.0,5.0,0.0,2.0,NaN,...,0.0,1.0,7.0,2.0,NaN,7.0,7.0,65.0,23,7
4825,E01033743,Greenwich 002H,Greenwich,1.0,22.0,3.0,3.0,1.0,8.0,NaN,...,1.0,2.0,15.0,6.0,NaN,23.0,29.0,301.0,183,18
4826,E01033744,Greenwich 007G,Greenwich,2.0,12.0,5.0,8.0,0.0,3.0,NaN,...,NaN,0.0,13.0,1.0,NaN,16.0,13.0,95.0,31,7
4827,E01033745,Greenwich 002I,Greenwich,1.0,22.0,4.0,7.0,1.0,2.0,NaN,...,0.0,1.0,11.0,8.0,NaN,22.0,10.0,133.0,46,12


In [15]:
#only shows the violent crime in dataset
historic_dfv = historic_df[(historic_df['Major Category'] == 'Violence Against the Person') | (historic_df['Major Category'] == 'Possession of Weapons')]
#sum the data
new_df2 = historic_dfv.groupby('LSOA Code')['crime2011'].sum().reset_index()
new_df2 = new_df2.rename(columns={'crime2011': 'ViolentCrime'})
new_df2

,LSOA Code,ViolentCrime
0,E01000006,17
1,E01000007,125
2,E01000008,33
3,E01000009,88
4,E01000010,230
...,...,...
4824,E01033742,15
4825,E01033743,53
4826,E01033744,29
4827,E01033745,32


In [16]:
#merge the violent data into the orginal data
rehistoric_df5 = pd.merge(rehistoric_df4,
                    new_df2,
                   left_on ='LSOA Code',
                   right_on='LSOA Code',
                   how='inner')
rehistoric_df5

,LSOA Code,LSOA Name,Borough,Arson and Criminal Damage: Arson,Arson and Criminal Damage: Criminal Damage,Burglary: Burglary Business and Community,Burglary: Domestic Burglary,Drug Offences: Drug Trafficking,Drug Offences: Possession of Drugs,Miscellaneous Crimes Against Society: Absconding from Lawful Custody,...,Vehicle Offences: Interfering with a Motor Vehicle,Vehicle Offences: Theft from a Motor Vehicle,Vehicle Offences: Theft or Taking of a Motor Vehicle,Violence Against the Person: Homicide,Violence Against the Person: Violence with Injury,Violence Against the Person: Violence without Injury,all_crime,PropertyCrime,antiCrime,ViolentCrime
0,E01000006,Barking and Dagenham 016A,Barking and Dagenham,0.0,4.0,1.0,21.0,1.0,9.0,NaN,...,2.0,16.0,13.0,NaN,10.0,7.0,109.0,43,14,17
1,E01000007,Barking and Dagenham 015A,Barking and Dagenham,2.0,39.0,5.0,8.0,2.0,50.0,0.0,...,0.0,28.0,7.0,0.0,66.0,53.0,426.0,135,89,125
2,E01000008,Barking and Dagenham 015B,Barking and Dagenham,1.0,14.0,4.0,9.0,2.0,18.0,NaN,...,1.0,25.0,14.0,NaN,12.0,20.0,179.0,61,30,33
3,E01000009,Barking and Dagenham 016B,Barking and Dagenham,3.0,34.0,9.0,14.0,2.0,54.0,0.0,...,4.0,23.0,13.0,0.0,44.0,42.0,351.0,109,77,88
4,E01000010,Barking and Dagenham 015C,Barking and Dagenham,3.0,69.0,33.0,18.0,0.0,102.0,NaN,...,1.0,46.0,13.0,0.0,96.0,122.0,1202.0,654,186,230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4824,E01033742,Greenwich 007F,Greenwich,1.0,9.0,1.0,5.0,0.0,2.0,NaN,...,1.0,7.0,2.0,NaN,7.0,7.0,65.0,23,7,15
4825,E01033743,Greenwich 002H,Greenwich,1.0,22.0,3.0,3.0,1.0,8.0,NaN,...,2.0,15.0,6.0,NaN,23.0,29.0,301.0,183,18,53
4826,E01033744,Greenwich 007G,Greenwich,2.0,12.0,5.0,8.0,0.0,3.0,NaN,...,0.0,13.0,1.0,NaN,16.0,13.0,95.0,31,7,29
4827,E01033745,Greenwich 002I,Greenwich,1.0,22.0,4.0,7.0,1.0,2.0,NaN,...,1.0,11.0,8.0,NaN,22.0,10.0,133.0,46,12,32


### housing price data 

In [17]:
house_price = pd.read_excel('../data/london-data-main/data/land-registry-house-prices-LSOA.xls',
                           sheet_name='Mean')

house_price = house_price[['Code','Year ending Mar 2011','Year ending Jun 2011','Year ending Sep 2011','Year ending Dec 2011']]

for var in ['Year ending Mar 2011','Year ending Jun 2011','Year ending Sep 2011','Year ending Dec 2011']:
    house_price[var] = pd.to_numeric(house_price[var],errors='coerce')
house_price['hp2011_mean']= house_price[['Year ending Mar 2011','Year ending Jun 2011','Year ending Sep 2011','Year ending Dec 2011']].mean(axis=1)

house_price=house_price[['Code','hp2011_mean']]

In [18]:
house_price

,Code,hp2011_mean
0,NaN,NaN
1,E01000001,5.613612e+05
2,E01000002,5.895431e+05
3,E01000003,3.973007e+05
4,E01000005,3.345166e+05
...,...,...
4831,E01033604,NaN
4832,E01033605,3.315291e+05
4833,E01033606,8.844827e+05
4834,E01033607,1.111940e+06


### age data

In [ ]:
age_ratio = pd.read_excel('../data/london-data-main/data/SAPE20DT12-mid-2012-lsoa-Broad_ages-estimates-formatted.xls',
                           sheet_name='Mid-2012 Persons',
                         skiprows = [0,1,2,3])
age_ratio = age_ratio[['Area Codes','All Ages','0-15','16-29','30-44','45-64','65+']]


males_ratio = pd.read_excel('../data/london-data-main/data/SAPE20DT12-mid-2012-lsoa-Broad_ages-estimates-formatted.xls',
                           sheet_name='Mid-2012 Males',
                         skiprows = [0,1,2,3])

males_ratio = males_ratio[['Area Codes','All Ages','0-15','16-29','30-44','45-64','65+']]

females_ratio = pd.read_excel('../data/london-data-main/data/SAPE20DT12-mid-2012-lsoa-Broad_ages-estimates-formatted.xls',
                           sheet_name='Mid-2012 Females',
                         skiprows = [0,1,2,3])

females_ratio = females_ratio[['Area Codes','All Ages','0-15','16-29','30-44','45-64','65+']]


In [ ]:
males_ratio

In [ ]:
## rename some variables to differentiate males and females 

var_names = ['All Ages','0-15','16-29','30-44','45-64','65+']

for age_range in var_names:
    males_ratio = males_ratio.rename(columns={age_range:age_range+'_male'})
    females_ratio = females_ratio.rename(columns={age_range:age_range+'_female'})

In [ ]:
## and merge all age ratios files 

age_ratio = pd.merge(age_ratio,
                    males_ratio,
                    left_on='Area Codes',
                    right_on='Area Codes',
                    how='outer')


age_ratio = pd.merge(age_ratio,
                    females_ratio,
                    left_on='Area Codes',
                    right_on='Area Codes',
                    how='outer')

In [ ]:
## caltulate a gender ratio 

for age_range in var_names:
    age_ratio[age_range+'_pm'] = age_ratio[age_range+'_male']*100/age_ratio[age_range+'_female']

In [ ]:
age_ratio['All Ages_pm'].describe()

In [ ]:
## calcualte age ratio 
for age_range in ['0-15','16-29','30-44','45-64','65+']:
    age_ratio[age_range] = age_ratio[age_range]*100/age_ratio['All Ages']
    

age_ratio = age_ratio.rename(columns = {'0-15':'age015',
                              '16-29':'age1629',
                              '30-44':'age3044',
                              '45-64':'age4564',
                              '65+':'age65'})



age_ratio = age_ratio.rename(columns = {
                        'All Ages_pm':'all_pm',
                            '0-15_pm':'age015_pm',
                              '16-29':'age1629_pm',
                              '30-44':'age3044_pm',
                              '45-64':'age4564_pm',
                              '65+':'age65_pm'})

In [ ]:
age_ratio

### CCTV 

In [ ]:
cctv = pd.read_excel('../data/london-data-main/data/CCTV monitoring number lsoa.xlsx')
cctv['Borough']=cctv['Borough'].str.replace("CCTV ", "")

In [ ]:
cctv.rename(columns={'Total Cameras (2012)':'totalcameras',
                    'Total Cameras (2022)':'2022_total_cameras'}, inplace=True)
cctv

### Unemployment

In [ ]:
unemployment = pd.read_excel('../data/london-data-main/data/Unemployment Borough.xlsx')
unemployment = unemployment.rename(columns={'Unemployment Rate':'ue_rate'})
unemployment

### The number of licensed retaurates-cafes-pubs-clubs borough

In [ ]:
rescafes = pd.read_excel('../data/london-data-main/data/restaurants_cafes.xlsx')
pubs = pd.read_excel('../data/london-data-main/data/pubs.xlsx')
clubs = pd.read_excel('../data/london-data-main/data/clubs.xlsx')
rescafes = rescafes.rename(columns={2011: 'restaurants_cafes'})
pubs = pubs.rename(columns = {2011:'pubs'})
clubs = clubs.rename(columns = {2011:'clubs'})

In [ ]:
clubs

In [ ]:
pubs_clubs_data = pd.merge(pubs,
                   clubs,
                   left_on='Borough',
                   right_on ='Borough',
                   how='inner')
pubs_clubs_rescafes_data = pd.merge(rescafes,
                   pubs_clubs_data,
                   left_on='Borough',
                   right_on ='Borough',
                   how='inner')
pubs_clubs_rescafes_data

In [ ]:
all_merge = pd.read_csv('../data/london-data-main/data/Merged_LSOA_Data.csv')

all_merge

### Merge multiple data 

In [ ]:
#
all_data = pd.merge(rehistoric_df5,
                    all_merge,
                   left_on ='LSOA Code',
                   right_on='LSOA11CD',
                   how='inner')

all_data = pd.merge(all_data,
                   house_price[['Code','hp2011_mean']],
                   left_on='LSOA Code',
                   right_on ='Code',
                   how='inner')


all_data = pd.merge(all_data,
                   age_ratio,
                   left_on='LSOA Code',
                   right_on ='Area Codes',
                   how='inner')


all_data = pd.merge(all_data,
                   cctv,
                   left_on='Borough',
                   right_on ='Borough',
                   how='inner')

all_data = pd.merge(all_data,
                   unemployment,
                   left_on='Borough',
                   right_on ='Borough',
                   how='inner')

all_data = pd.merge(all_data,
                   pubs_clubs_rescafes_data,
                   left_on='Borough',
                   right_on ='Borough',
                   how='inner')

In [ ]:
## see all variables 
all_data['geometry']

### Rearrange

In [ ]:
df = pd.read_csv('../data/london-data-main/data/lsoa_IMD_airbnb_housing.csv')

In [ ]:
#check the data
df[['geometry']][df['LSOA code'] == 'E01000006']

In [ ]:
for i in range(len(all_data)):
    lsoacode = all_data.loc[i]['LSOA Code']
    real_geometry = df['geometry'][df['LSOA code'] == lsoacode]
    all_data.loc[i, 'geometry'] = list(real_geometry.items())[0][1]
    #print(all_data.loc[i]['geometry'])

In [ ]:
all_data[['geometry']][all_data['LSOA Code'] == 'E01000006']

In [ ]:
all_data.to_csv('merge_crime and lsoa data.csv', index=False)

In [ ]:
all_data

### See the correlation between average crime and other variables 

In [ ]:
all_data[['Code','hp2011_mean','totalcameras', 'age1629']].dtypes

In [ ]:
all_data[['Arson and Criminal Damage: Criminal Damage',
          'PM10max']].corr()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
## create some log variables 

all_data['lhp2011_mean'] = np.log(all_data['hp2011_mean'])

all_data['lHHI_mean'] = np.log(all_data['HHI_mean'])

all_data['lPOPDEN'] = np.log(all_data['POPDEN'])

In [ ]:
all_data[['hp2011_mean']]

In [ ]:
plt.hist(all_data[['HHI_mean']], bins=30, edgecolor='black')

In [ ]:
plt.hist(all_data[['lHHI_mean']], bins=30, edgecolor='black')

plt.title('Ditribution of Average Income Dataset')

In [ ]:
plt.hist(all_data[['POPDEN']], bins=30, edgecolor='black')
plt.title('Ditribution of Population Density Dataset')

In [ ]:
plt.hist(all_data[['lPOPDEN']], bins=30, edgecolor='black')

from scipy.stats import skew
data_skewness = skew(all_data[['hp2011_mean']])
all_data.dropna(inplace=True)
#print("Skewness:", data_skewness)

In [ ]:
## normalize crime 

## all variables below have the units being percentage points 

all_data['crime_rate'] = all_data['all_crime']*100/all_data['USUALRES']
all_data['pcrime_rate'] = all_data['PropertyCrime']*100/all_data['USUALRES']
all_data['acrime_rate'] = all_data['antiCrime']*100/all_data['USUALRES']
all_data['vcrime_rate'] = all_data['ViolentCrime']*100/all_data['USUALRES']
all_data['restaurants_cafes_rate'] = all_data['restaurants_cafes']*100/all_data['USUALRES']
all_data['pubs_rate'] = all_data['pubs']*100/all_data['USUALRES']
all_data['clubs_rate'] = all_data['clubs']*100/all_data['USUALRES']


In [ ]:
import seaborn as sb
  
#plotting correlation heatmap
dataplot = sb.heatmap(all_data[['crime_rate',
                                'pcrime_rate',
                             'lHHI_mean',
                             'age1629',
                              'all_pm',
                             'lPOPDEN',
                             'totalcameras',
                            'clubs_rate',
                            'PM25mean']].corr(), 
                      cmap="YlGnBu", 
                      annot=True)

In [ ]:
## create some log variables 

all_data['lhp2011_mean'] = np.log(all_data['hp2011_mean'])

all_data['lHHI_mean'] = np.log(all_data['HHI_mean'])

all_data['lPOPDEN'] = np.log(all_data['POPDEN'])

In [ ]:
variables = ['crime_rate',
             'pcrime_rate',
             'vcrime_rate',
             'lHHI_mean',
             'age1629',
             'all_pm',
             'lPOPDEN',
             'totalcameras',
            'clubs_rate'] ## add more variables if I like 

summary_table = all_data[variables].describe().T
summary_table

### regression analysis for all crime

In [ ]:
all_data['crime_rate'].describe

In [ ]:
## economic factors
#lm_1 = sm.OLS.from_formula('crime_rate ~ lHHI_mean + ue_rate', data= all_data).fit()
 
## economic factors 
lm_1 = sm.OLS.from_formula('crime_rate ~  lHHI_mean + ue_rate + C(Borough)', data= all_data).fit()

## economic factors + demographic factors 
lm_2 = sm.OLS.from_formula('crime_rate ~   lHHI_mean + ue_rate + age1629  + lPOPDEN+C(Borough)', data= all_data).fit()

lm_3 = sm.OLS.from_formula('crime_rate ~  lHHI_mean + ue_rate + age1629+ all_pm + lPOPDEN+C(Borough)', data= all_data).fit()

## economic factors + demographic factors + public security 
lm_4 = sm.OLS.from_formula('crime_rate ~  lHHI_mean + age1629 + lPOPDEN + totalcameras', data= all_data).fit()

## economic factors + demographic factors +  public security  + social contexts/environment 

lm_5 = sm.OLS.from_formula('crime_rate ~  lHHI_mean + age1629 + lPOPDEN + totalcameras + clubs_rate', data= all_data).fit()

## economic factors + demographic factors +  public security  + social contexts/environment + air polution 
#lm_5 = sm.OLS.from_formula('crime_rate ~  age1629 + lPOPDEN +PM25mean+C(Borough)', data= all_data).fit()

regression_table = Stargazer((lm_1,
                    lm_2,
                   lm_3,
                    lm_4,
                    lm_5))


regression_table.add_line('Borough Fixed Effect',
                        ['Yes',
                        'Yes',
                        'Yes',
                        'No',
                        'No'])

regression_table.show_f_statistic=False
from IPython.core.display import HTML
HTML(regression_table.render_html())

In [ ]:
## take log on variables is to make the coefficients insensitive to the units of measure/ normalize the change retative to its value
## /descale it 


## each percent increase in HHI_mean (1% increase): 2 percentage points decrease in crime rate, e.g. 4% -> 2%
## each percentage point increase in UE_rate  (3% -> 4%): 1.4 percentage points increase in crime rate, e.g. 3% ->4%.
### each 1% increase in population density: 4.9 percentage points decrease in crime rate, e.g. 5.9% -> 1%


In [ ]:
#1. interaction effect/neighbourhood effect/endogeneous effect/peer effect: Y1 and Y2 and Yn have effects on each other
#2. common factor/ common driver:  X1 and X2 are similar to each other and X affects Y. hence Y1 and Y2 are correlated. 

### regression analysis for property crime

In [ ]:
all_data['pcrime_rate'].describe

In [ ]:
## economic factors
#lm_1 = sm.OLS.from_formula('crime_rate ~ lHHI_mean + ue_rate', data= all_data).fit()
 
## economic factors 
lm_1 = sm.OLS.from_formula('pcrime_rate ~  lHHI_mean + ue_rate + C(Borough)', data= all_data).fit()

## economic factors + demographic factors 
#lm_2 = sm.OLS.from_formula('pcrime_rate ~  lHHI_mean + ue_rate + age1629  + lPOPDEN+C(Borough)', data= all_data).fit()

lm_2 = sm.OLS.from_formula('pcrime_rate ~  lHHI_mean + ue_rate + age1629+ all_pm + lPOPDEN+C(Borough)', data= all_data).fit()

## economic factors + demographic factors + public security 
lm_3 = sm.OLS.from_formula('pcrime_rate ~  lHHI_mean + ue_rate + age1629 + all_pm + lPOPDEN + totalcameras', data= all_data).fit()

## economic factors + demographic factors +  public security  + social contexts/environment 

lm_4 = sm.OLS.from_formula('pcrime_rate ~ lHHI_mean + ue_rate + age1629 + all_pm + lPOPDEN + totalcameras + clubs_rate', data= all_data).fit()

## economic factors + demographic factors +  public security  + social contexts/environment + air polution 
#lm_5 = sm.OLS.from_formula('crime_rate ~  age1629 + lPOPDEN +PM25mean+C(Borough)', data= all_data).fit()

regression_table = Stargazer((lm_1,
                    lm_2,
                   lm_3,
                    lm_4
                    #lm_5
                             ))


regression_table.add_line('Borough Fixed Effect',
                        ['Yes',
                        'Yes',
                        'No',
                        'No'])

regression_table.show_f_statistic=False
from IPython.core.display import HTML
HTML(regression_table.render_html())

### regression analysis for antisocial behaviour crime

In [ ]:
all_data['acrime_rate'].describe

In [ ]:
## economic factors
#lm_1 = sm.OLS.from_formula('crime_rate ~ lHHI_mean + ue_rate', data= all_data).fit()
 
## economic factors 
lm_1 = sm.OLS.from_formula('acrime_rate ~  lHHI_mean + ue_rate + C(Borough)', data= all_data).fit()

## economic factors + demographic factors 
lm_2 = sm.OLS.from_formula('acrime_rate ~   lHHI_mean + ue_rate + age1629  + lPOPDEN+C(Borough)', data= all_data).fit()

lm_3 = sm.OLS.from_formula('acrime_rate ~  lHHI_mean + ue_rate + age1629+all_pm + lPOPDEN+C(Borough)', data= all_data).fit()

## economic factors + demographic factors + public security 
lm_4 = sm.OLS.from_formula('acrime_rate ~  lHHI_mean + age1629 + lPOPDEN + totalcameras', data= all_data).fit()

## economic factors + demographic factors +  public security  + social contexts/environment 

lm_5 = sm.OLS.from_formula('acrime_rate ~  lHHI_mean + age1629 + lPOPDEN + totalcameras + clubs_rate', data= all_data).fit()

## economic factors + demographic factors +  public security  + social contexts/environment + air polution 
#lm_5 = sm.OLS.from_formula('crime_rate ~  age1629 + lPOPDEN +PM25mean+C(Borough)', data= all_data).fit()

regression_table = Stargazer((lm_1,
                    lm_2,
                   lm_3,
                    lm_4,
                    lm_5))


regression_table.add_line('Borough Fixed Effect',
                        ['Yes',
                        'Yes',
                        'Yes',
                        'No',
                        'No'])

regression_table.show_f_statistic=False
from IPython.core.display import HTML
HTML(regression_table.render_html())

### regression analysis for violent crime

In [ ]:
all_data['vcrime_rate'].describe

In [ ]:
## economic factors
#lm_1 = sm.OLS.from_formula('crime_rate ~ lHHI_mean + ue_rate', data= all_data).fit()
 
## economic factors 
lm_1 = sm.OLS.from_formula('vcrime_rate ~  lHHI_mean + ue_rate + C(Borough)', data= all_data).fit()

## economic factors + demographic factors 
lm_2 = sm.OLS.from_formula('vcrime_rate ~   lHHI_mean + ue_rate + age1629  + lPOPDEN+C(Borough)', data= all_data).fit()

lm_3 = sm.OLS.from_formula('vcrime_rate ~  lHHI_mean + ue_rate + age1629+ all_pm + lPOPDEN+C(Borough)', data= all_data).fit()

## economic factors + demographic factors + public security 
lm_4 = sm.OLS.from_formula('vcrime_rate ~  lHHI_mean + age1629 + lPOPDEN + totalcameras', data= all_data).fit()

## economic factors + demographic factors +  public security  + social contexts/environment 

lm_5 = sm.OLS.from_formula('vcrime_rate ~  lHHI_mean + age1629 + lPOPDEN + totalcameras + clubs_rate', data= all_data).fit()

## economic factors + demographic factors +  public security  + social contexts/environment + air polution 
#lm_5 = sm.OLS.from_formula('crime_rate ~  age1629 + lPOPDEN +PM25mean+C(Borough)', data= all_data).fit()

regression_table = Stargazer((lm_1,
                    lm_2,
                   lm_3,
                    lm_4,
                    lm_5))


regression_table.add_line('Borough Fixed Effect',
                        ['Yes',
                        'Yes',
                        'Yes',
                        'No',
                        'No'])

regression_table.show_f_statistic=False
from IPython.core.display import HTML
HTML(regression_table.render_html())

In [ ]:
## violent crime rate
## economic factors 
lm_1 = sm.OLS.from_formula('vcrime_rate ~  lHHI_mean + ue_rate + C(Borough)', data= all_data).fit()

## economic factors + demographic factors 

lm_2 = sm.OLS.from_formula('vcrime_rate ~  lHHI_mean + ue_rate + age1629+ all_pm + lPOPDEN+C(Borough)', data= all_data).fit()

## economic factors + demographic factors +  public security  + social contexts/environment 

lm_3 = sm.OLS.from_formula('vcrime_rate ~ lHHI_mean + ue_rate + age1629 + all_pm + lPOPDEN + totalcameras + clubs_rate', data= all_data).fit()

## antisocial behaviour crime rate
## economic factors 
lm_4 = sm.OLS.from_formula('acrime_rate ~  lHHI_mean + ue_rate + C(Borough)', data= all_data).fit()

## economic factors + demographic factors 

lm_5 = sm.OLS.from_formula('acrime_rate ~  lHHI_mean + ue_rate + age1629+ all_pm + lPOPDEN+C(Borough)', data= all_data).fit()

## economic factors + demographic factors +  public security  + social contexts/environment 

lm_6 = sm.OLS.from_formula('acrime_rate ~ lHHI_mean + ue_rate + age1629 + all_pm + lPOPDEN + totalcameras + clubs_rate', data= all_data).fit()


regression_table = Stargazer((lm_1,
                    lm_2,
                   lm_3,
                    lm_4,
                    lm_5,
                    lm_6))


regression_table.add_line('Borough Fixed Effect',
                        ['Yes',
                        'Yes',
                        'No',
                        'Yes',
                        'Yes',
                        'No'])

regression_table.show_f_statistic=False
from IPython.core.display import HTML
HTML(regression_table.render_html())

### Spatial Weight

In [ ]:
import sys
import os
import urllib
import zipfile
import geopandas as gpd
import seaborn as sns
import shapely.geometry
from shapely.geometry import Point, Polygon
import libpysal as lps
from libpysal.weights import Queen, Rook, KNN
import pysal as ps
import numpy as np
import pandas as pd
import spreg
import mgwr
import mapclassify
import matplotlib.pyplot as plt
import pysal.viz as viz
%matplotlib inline
import warnings
warnings.simplefilter('ignore')
from shapely.wkt import loads 
from shapely import wkt
from libpysal import weights

In [ ]:
df = pd.read_csv('merge_crime and lsoa data.csv')

In [ ]:
df['crime_rate'] = df['all_crime']*100/all_data['USUALRES']
df['pcrime_rate'] = df['PropertyCrime']*100/all_data['USUALRES']
df['acrime_rate'] = df['antiCrime']*100/all_data['USUALRES']
df['vcrime_rate'] = df['ViolentCrime']*100/all_data['USUALRES']

In [ ]:
df = df[['LSOA Code','LSOA Name','Borough','HHI_mean','crime_rate','pcrime_rate','acrime_rate','vcrime_rate','age1629','hp2011_mean','totalcameras','PM25mean','POPDEN','restaurants_cafes','pubs','clubs','geometry']]
df.info()

In [ ]:
df = df.dropna()
df.info()

In [ ]:
df['geometry'] = df['geometry'].apply(wkt.loads)

In [ ]:
crs = {'init': 'epsg:27700'}
gdf = gpd.GeoDataFrame(df, crs=crs, geometry='geometry')

In [ ]:
gdf.to_file('your_shapefile.shp', driver='ESRI Shapefile')

In [ ]:
gdf=gpd.read_file('your_shapefile.shp')
gdf

In [ ]:
gdf.info()

In [ ]:
if gdf.crs is not None:
    print('CRS information is present.')
else:
    print('CRS information is not present.')

In [ ]:
gdf.crs

In [ ]:
print(gdf.columns.tolist())

In [ ]:
w_queen = Queen.from_dataframe(gdf)
w_queen.n

In [ ]:
print ('%.4f'%w_queen.pct_nonzero)

In [ ]:
w_queen.histogram

In [ ]:
w_queen.neighbors

In [ ]:
gdf.plot(column='crime_rate', 
         alpha=0.8, 
         cmap='coolwarm', 
         scheme='quantiles',
        legend = True)

### Spatial Autocorrelation: Moran's I

In [ ]:
# Graphics
import matplotlib.pyplot as plt
from matplotlib import colors
# Analysis
import geopandas as gpd
import pandas as pd
from libpysal import weights
from pysal.explore import esda
import numpy as np

In [ ]:
gdf.head()

In [ ]:
#all crime
fig, ax = plt.subplots(1, figsize=(14, 14))
gdf.plot(column='crime_rate', cmap='viridis',
scheme='quantiles', k=5,
linewidth=0.,
legend=True, legend_kwds={"title":"Crime Rate (%)","loc": 2},
ax=ax)
ax.set_axis_off()

In [ ]:
w = weights.KNN.from_dataframe(gdf, k=8)
w.transform = 'R'
moran = esda.moran.Moran(gdf['crime_rate'], w)
round(moran.I,3)

In [ ]:
moran.p_sim

In [ ]:
#property crime
fig, ax = plt.subplots(1, figsize=(14, 14))
gdf.plot(column='pcrime_rat', cmap='viridis',
scheme='quantiles', k=5,
linewidth=0.,
legend=True, legend_kwds={"title":"Proper Crime Rate (%)","loc": 2},
ax=ax)
ax.set_axis_off()

In [ ]:
w = weights.KNN.from_dataframe(gdf, k=8)
w.transform = 'R'
moran = esda.moran.Moran(gdf['pcrime_rat'], w)
round(moran.I,3)

In [ ]:
moran.p_sim

In [ ]:
#antisocial behaviour crime
fig, ax = plt.subplots(1, figsize=(14, 14))
gdf.plot(column='acrime_rat', cmap='viridis',
scheme='quantiles', k=5,
linewidth=0.,
legend=True, legend_kwds={"title":"Antisocial Behaviour Crime Rate (%)","loc": 2},
ax=ax)
ax.set_axis_off()

In [ ]:
w = weights.KNN.from_dataframe(gdf, k=8)
w.transform = 'R'
moran = esda.moran.Moran(gdf['acrime_rat'], w)
round(moran.I,3)

In [ ]:
moran.p_sim

In [ ]:
#violent crime
fig, ax = plt.subplots(1, figsize=(14, 14))
gdf.plot(column='vcrime_rat', cmap='viridis',
scheme='quantiles', k=5,
linewidth=0.,
legend=True, legend_kwds={"title":"Violent Crime Rate (%)","loc": 2},
ax=ax)
ax.set_axis_off()

In [ ]:
w = weights.KNN.from_dataframe(gdf, k=8)
w.transform = 'R'
moran = esda.moran.Moran(gdf['vcrime_rat'], w)
round(moran.I,3)

In [ ]:
moran.p_sim

### Moran Plot for crime rate

In [ ]:
gdf['crime_rate_lag'] = weights.spatial_lag.lag_spatial(w, gdf['crime_rate'])

In [ ]:
def standardize(df, var):
    name = var + '_z'
    df[name] = (df[var] - df[var].mean()) / df[var].std()

standardize(gdf,'crime_rate')
standardize(gdf,'crime_rate_lag')

In [ ]:
gdf.tail()

In [ ]:
fig, ax = plt.subplots(1, figsize=(14, 14))
gdf.plot(column='crime_rate_lag', cmap='viridis',
scheme='quantiles', k=5,
linewidth=0.,
legend=True, legend_kwds={"title":"Crime Rate (%)","loc": 2},
ax=ax)
ax.set_axis_off()

In [ ]:
# Setup the figure and axis
fig, ax = plt.subplots(1, figsize=(6, 6))
# Plot values
plt.scatter(gdf['crime_rate_z'], gdf['crime_rate_lag_z'])
# Display
ax.set_title('Moran Plot (% crime rate)')
ax.set_xlabel("crime rate % (z)")
ax.set_ylabel("Neighbours' Mean crime rate % (z)")
plt.show()

In [ ]:
# Setup the figure and axis
fig, ax = plt.subplots(1, figsize=(6, 6))
# Plot values
plt.scatter(gdf['crime_rate_z'], gdf['crime_rate_lag_z'])
# Add vertical and horizontal lines through zero
ax.axvline(0, c='k', alpha=0.5)
ax.axhline(0, c='k', alpha=0.5)
# Add text labels for each quadrant
plt.text(0.75, 1.5, "HH", fontsize=25)
plt.text(0.75, -3, "HL", fontsize=25)
plt.text(-1.75, 1.5, "LH", fontsize=25)
plt.text(-1.75, -3, "LL", fontsize=25)
# Display
ax.set_title('Moran Plot (% crime rate)')
ax.set_xlabel("crime rate % (z)")
ax.set_ylabel("Local Mean crime rate % (z)")
plt.show()

In [ ]:
def rules(row):
    if row['crime_rate_z'] > 0:
        if row['crime_rate_lag_z'] > 0:
            return 'HH'
        else:
            return 'HL'
    else:
        if row['crime_rate_lag_z'] > 0:
            return 'LH'
        else:
            return 'LL'

gdf['quadrant'] = gdf.apply(rules, 1)

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,8))

qcolors = {'HH':'red', 'HL':'pink', 'LH':'lightblue', 'LL':'blue'}

#map
gdf.plot(column='quadrant', categorical=True, cmap=colors.ListedColormap(qcolors.values()),
         k=2, edgecolor='white', linewidth=0.0,
         legend=True, legend_kwds={"title":'Crime Rate Moran I Quadrant',"loc": 2},
         ax=ax)

plt.show()

### Moran I for Property Crime

In [ ]:
gdf['pcrime_rat_lag'] = weights.spatial_lag.lag_spatial(w, gdf['pcrime_rat'])

In [ ]:
def standardize(df, var):
    name = var + '_z'
    df[name] = (df[var] - df[var].mean()) / df[var].std()

standardize(gdf,'pcrime_rat')
standardize(gdf,'pcrime_rat_lag')

In [ ]:
def rules(row):
    if row['pcrime_rat_z'] > 0:
        if row['pcrime_rat_lag_z'] > 0:
            return 'HH'
        else:
            return 'HL'
    else:
        if row['pcrime_rat_lag_z'] > 0:
            return 'LH'
        else:
            return 'LL'

gdf['quadrant'] = gdf.apply(rules, 1)

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,8))

qcolors = {'HH':'red', 'HL':'pink', 'LH':'yellow', 'LL':'blue'}

#map
gdf.plot(column='quadrant', categorical=True, cmap=colors.ListedColormap(qcolors.values()),
         k=2, edgecolor='white', linewidth=0.0,
         legend=True, legend_kwds={"title":'LISA (Moran I) for Property Crime',"loc": 2},
         ax=ax)


### Moran I for antisocial behaviour crime

In [ ]:
gdf['acrime_rat_lag'] = weights.spatial_lag.lag_spatial(w, gdf['acrime_rat'])

In [ ]:
def standardize(df, var):
    name = var + '_z'
    df[name] = (df[var] - df[var].mean()) / df[var].std()

standardize(gdf,'acrime_rat')
standardize(gdf,'acrime_rat_lag')

In [ ]:
def rules(row):
    if row['acrime_rat_z'] > 0:
        if row['acrime_rat_lag_z'] > 0:
            return 'HH'
        else:
            return 'HL'
    else:
        if row['acrime_rat_lag_z'] > 0:
            return 'LH'
        else:
            return 'LL'

gdf['quadrant'] = gdf.apply(rules, 1)

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,8))

qcolors = {'HH':'red', 'HL':'pink', 'LH':'yellow', 'LL':'blue'}

#map
gdf.plot(column='quadrant', categorical=True, cmap=colors.ListedColormap(qcolors.values()),
         k=2, edgecolor='white', linewidth=0.0,
         legend=True, legend_kwds={"title":'Antisocial Behaviour Crime Moran I Quadrant',"loc": 2},
         ax=ax)

### Moran I for violent crime

In [ ]:
gdf['vcrime_rat_lag'] = weights.spatial_lag.lag_spatial(w, gdf['vcrime_rat'])

In [ ]:
def standardize(df, var):
    name = var + '_z'
    df[name] = (df[var] - df[var].mean()) / df[var].std()

standardize(gdf,'vcrime_rat')
standardize(gdf,'vcrime_rat_lag')

In [ ]:
def rules(row):
    if row['vcrime_rat_z'] > 0:
        if row['vcrime_rat_lag_z'] > 0:
            return 'HH'
        else:
            return 'HL'
    else:
        if row['vcrime_rat_lag_z'] > 0:
            return 'LH'
        else:
            return 'LL'

gdf['quadrant'] = gdf.apply(rules, 1)

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,8))

qcolors = {'HH':'red', 'HL':'pink', 'LH':'yellow', 'LL':'blue'}

#map
gdf.plot(column='quadrant', categorical=True, cmap=colors.ListedColormap(qcolors.values()),
         k=2, edgecolor='white', linewidth=0.0,
         legend=True, legend_kwds={"title":'Violent Crime Moran I Quadrant',"loc": 2},
         ax=ax)

### Local Moran's I for property crime rate

In [ ]:
lisa = esda.moran.Moran_Local(gdf['pcrime_rat'], w)

In [ ]:
gdf['p-sim'] = lisa.p_sim
sig = 1 * (lisa.p_sim < 0.05)
slabels = ['non-sig.', 'significant']
labels = [slabels[i] for i in sig]
gdf['sig'] = labels
gdf[['sig','p-sim']].head(10)


In [ ]:
fig, ax = plt.subplots(1,figsize=(10,8))
sigcolors = {'non-sig.':'lightgrey', 'significant':'black'}
gdf.plot(column='sig', categorical=True, cmap=colors.ListedColormap(sigcolors.values()),
k=2, linewidth=0.1, edgecolor='white',
legend=True, legend_kwds={"title":'Local Moran I for Property Crime Rate',"loc": 2},
ax=ax)
plt.show()


In [ ]:
lisa.q[1:10]

In [ ]:
counts = [(j,(lisa.q==j).sum()) for j in range(1,5)]
counts

In [ ]:
qlabels = ['HH', 'LH', 'LL', 'HL'] #pysal scheme is HH=1, LH=2, LL=3, HL=4
labels = [qlabels[i-1] for i in lisa.q] #list substituting 1-4 with HH-HL
labels[1:10]


In [ ]:
gdf['qlabels'] = labels
[(qlabel, (gdf['qlabels']==qlabel).sum()) for qlabel in qlabels]


In [ ]:
hotspot = 1 * (sig * lisa.q==1)
coldspot = 3 * (sig * lisa.q==3)
doughnut = 2 * (sig * lisa.q==2)
diamond = 4 * (sig * lisa.q==4)
spots = hotspot + coldspot + doughnut + diamond

spot_labels = [ '0 non-sig.', '1 HH-sig.', '2 LH-sig.', '3 LL-sig.', '4 HL-sig.']
labels = [spot_labels[i] for i in spots]

In [ ]:
gdf['slabels'] = labels
[(spot_label, (gdf['slabels']==spot_label).sum()) for spot_label in spot_labels]

In [ ]:
fig, ax = plt.subplots(1, figsize=(14,14))
sigcolors = colors.ListedColormap([ 'lightgrey', 'red', 'yellow', 'blue', 'pink'])
gdf.plot(column='slabels', categorical=True, 
         k=2, cmap=sigcolors, linewidth=0.1, edgecolor='white', 
         legend=True, legend_kwds={"title":"LISA (Moran's I) for property crime","loc": 2}, 
         ax=ax)
plt.show()


### Local Moran's I for violent crime rate

In [ ]:
lisa = esda.moran.Moran_Local(gdf['vcrime_rat'], w)

In [ ]:
def standardize(df, var):
    name = var + '_z'
    df[name] = (df[var] - df[var].mean()) / df[var].std()

standardize(gdf,'vcrime_rat')
standardize(gdf,'vcrime_rat_lag')

In [ ]:
gdf['p-sim'] = lisa.p_sim
sig = 1 * (lisa.p_sim < 0.05)
slabels = ['non-sig.', 'significant']
labels = [slabels[i] for i in sig]
gdf['sig'] = labels
gdf[['sig','p-sim']].head(10)

In [ ]:
fig, ax = plt.subplots(1,figsize=(10,8))
sigcolors = {'non-sig.':'lightgrey', 'significant':'black'}
gdf.plot(column='sig', categorical=True, cmap=colors.ListedColormap(sigcolors.values()),
k=2, linewidth=0.1, edgecolor='white',
legend=True, legend_kwds={"title":'Local Moran I for Property Crime Rate',"loc": 2},
ax=ax)
plt.show()

In [ ]:
lisa.q[1:10]

In [ ]:
counts = [(j,(lisa.q==j).sum()) for j in range(1,5)]
counts

In [ ]:
qlabels = ['HH', 'LH', 'LL', 'HL'] #pysal scheme is HH=1, LH=2, LL=3, HL=4
labels = [qlabels[i-1] for i in lisa.q] #list substituting 1-4 with HH-HL
labels[1:10]

In [ ]:
gdf['qlabels'] = labels
[(qlabel, (gdf['qlabels']==qlabel).sum()) for qlabel in qlabels]

In [ ]:
hotspot = 1 * (sig * lisa.q==1)
coldspot = 3 * (sig * lisa.q==3)
doughnut = 2 * (sig * lisa.q==2)
diamond = 4 * (sig * lisa.q==4)
spots = hotspot + coldspot + doughnut + diamond

spot_labels = [ '0 non-sig.', '1 HH-sig.', '2 LH-sig.', '3 LL-sig.', '4 HL-sig.']
labels = [spot_labels[i] for i in spots]

In [ ]:
gdf['slabels'] = labels
[(spot_label, (gdf['slabels']==spot_label).sum()) for spot_label in spot_labels]

In [ ]:
fig, ax = plt.subplots(1, figsize=(14,14))
sigcolors = colors.ListedColormap([ 'lightgrey', 'red', 'yellow', 'blue', 'pink'])
gdf.plot(column='slabels', categorical=True, 
         k=2, cmap=sigcolors, linewidth=0.1, edgecolor='white', 
         legend=True, legend_kwds={"title":"LISA (Moran's I) for violent crime rate","loc": 2}, 
         ax=ax)
plt.show()

### Local Moran's I for antisocial behaviour crime rate

In [ ]:
lisa = esda.moran.Moran_Local(gdf['acrime_rat'], w)

In [ ]:
def standardize(df, var):
    name = var + '_z'
    df[name] = (df[var] - df[var].mean()) / df[var].std()

standardize(gdf,'vcrime_rat')
standardize(gdf,'vcrime_rat_lag')

In [ ]:
gdf['p-sim'] = lisa.p_sim
sig = 1 * (lisa.p_sim < 0.05)
slabels = ['non-sig.', 'significant']
labels = [slabels[i] for i in sig]
gdf['sig'] = labels
gdf[['sig','p-sim']].head(10)

In [ ]:
fig, ax = plt.subplots(1,figsize=(10,8))
sigcolors = {'non-sig.':'lightgrey', 'significant':'black'}
gdf.plot(column='sig', categorical=True, cmap=colors.ListedColormap(sigcolors.values()),
k=2, linewidth=0.1, edgecolor='white',
legend=True, legend_kwds={"title":'Local Moran I for Property Crime Rate',"loc": 2},
ax=ax)
plt.show()

In [ ]:
lisa.q[1:10]

In [ ]:
counts = [(j,(lisa.q==j).sum()) for j in range(1,5)]
counts

In [ ]:
qlabels = ['HH', 'LH', 'LL', 'HL'] #pysal scheme is HH=1, LH=2, LL=3, HL=4
labels = [qlabels[i-1] for i in lisa.q] #list substituting 1-4 with HH-HL
labels[1:10]

In [ ]:
gdf['qlabels'] = labels
[(qlabel, (gdf['qlabels']==qlabel).sum()) for qlabel in qlabels]

In [ ]:
hotspot = 1 * (sig * lisa.q==1)
coldspot = 3 * (sig * lisa.q==3)
doughnut = 2 * (sig * lisa.q==2)
diamond = 4 * (sig * lisa.q==4)
spots = hotspot + coldspot + doughnut + diamond

spot_labels = [ '0 non-sig.', '1 HH-sig.', '2 LH-sig.', '3 LL-sig.', '4 HL-sig.']
labels = [spot_labels[i] for i in spots]

In [ ]:
gdf['slabels'] = labels
[(spot_label, (gdf['slabels']==spot_label).sum()) for spot_label in spot_labels]

In [ ]:
fig, ax = plt.subplots(1, figsize=(14,14))
sigcolors = colors.ListedColormap([ 'lightgrey', 'red', 'yellow', 'blue', 'pink'])
gdf.plot(column='slabels', categorical=True, 
         k=2, cmap=sigcolors, linewidth=0.1, edgecolor='white', 
         legend=True, legend_kwds={"title":"LISA (Moran's I) for anticosial behaviour crime rate","loc": 2}, 
         ax=ax)
plt.show()